## NNCL ASSIGNMENT 3

In [8]:
import numpy as np
import pandas as pd

In [14]:
class Network:
    def __init__(self, P, K, learning_rate):
        # P = amount of input samples
        self.P = P
        # K = amount of neurons in the hidden layer
        self.K = K
        # eta = learning rate
        self.eta = learning_rate
        # W = weights form input to hidden layers
        self.W = self.initWeights1()
        # V = weights from hidden layer to output, fixed to 1
        self.V = np.ones((1, self.K))

    def initWeights1(self):
        
        # Generate random vectors and normalize each vector to have a norm of 1
        weights = np.random.randn(self.P, self.K)
        norms_squared = np.linalg.norm(weights, axis=1, keepdims=True)**2
        normalized_weights = weights / norms_squared

        return normalized_weights
    
    def forwardPass(self, x):
        """
        Tanh activation function. 
        """
        
        # Calculate the dot product of the first-layer weights and the input
        dot_product = np.dot(self.W, x)
        # Apply hyperbolic tangent element-wise and sum for sigma. 
        tanh_result = np.tanh(dot_product)
        sigma = np.sum(self.V * tanh_result)

        # Sigma is the output of the network for a given input x
        return sigma

    def stochasticGradientDescent(self, sigma, tau):
        """
        Stochastic gradient descent
        """
        # Error is the quadratic difference between sigma (network output) and
        # tau (target value)
        error = ((sigma - tau)**2)/2
        # Use the gradient with respect to its contribution to the error
        gradient = (sigma - tau) * (1 - sigma**2)
        # Update the weights
        self.W = self.W - self.eta * gradient * error
        return error


    def train(self, t_max, dataset):
        """
        Train the network using stochastic gradient descent. 
        """
        # Select a random sample from the dataset, and perform a forward pass.
        # Then, update the weights using the SGD algorithm.
        # Run for t_max * P iterations.
        # Select a random sample from the dataset, make sure that for each t,
        # all samples are used, but in random order.
        for epoch in range(t_max):
            # For each epoch, keep track of the error and print the average
            # error for the epoch.
            epoch_error = 0
            for p in np.random.permutation(self.P):
                xi, tau = dataset[p]
                sigma = self.forwardPass(xi)
                error = self.stochasticGradientDescent(sigma, tau)
                epoch_error += error
            epoch_error /= self.P
            print("Epoch: {}, Error: {}".format(epoch, epoch_error))


# Inputs
xi = pd.read_csv("data/xi.csv", delimiter=',', header=None)
# Labels
tau = pd.read_csv("data/tau.csv", delimiter=',', header=None)
dataset = list(zip(xi, tau))

# n = amount of input samples
P = len(xi)
K = 2

network = Network(P=P, K=K, learning_rate=0.05)
network.train(t_max=100, dataset=dataset)

Epoch: 0, Error: 7633.616116569289
Epoch: 1, Error: 7754.25
Epoch: 2, Error: 7754.25
Epoch: 3, Error: 7754.25
Epoch: 4, Error: 7754.25
Epoch: 5, Error: 7754.25
Epoch: 6, Error: 7754.25
Epoch: 7, Error: 7754.25
Epoch: 8, Error: 7754.25
Epoch: 9, Error: 7754.25
Epoch: 10, Error: 7754.25
Epoch: 11, Error: 7754.25
Epoch: 12, Error: 7754.25
Epoch: 13, Error: 7754.25
Epoch: 14, Error: 7754.25
Epoch: 15, Error: 7754.25
Epoch: 16, Error: 7754.25
Epoch: 17, Error: 7754.25
Epoch: 18, Error: 7754.25
Epoch: 19, Error: 7754.25
Epoch: 20, Error: 7754.25
Epoch: 21, Error: 7754.25
Epoch: 22, Error: 7754.25
Epoch: 23, Error: 7754.25
Epoch: 24, Error: 7754.25
Epoch: 25, Error: 7754.25
Epoch: 26, Error: 7754.25
Epoch: 27, Error: 7754.25
Epoch: 28, Error: 7754.25
Epoch: 29, Error: 7754.25
Epoch: 30, Error: 7754.25
Epoch: 31, Error: 7754.25
Epoch: 32, Error: 7754.25
Epoch: 33, Error: 7754.25
Epoch: 34, Error: 7754.25
Epoch: 35, Error: 7754.25
Epoch: 36, Error: 7754.25
Epoch: 37, Error: 7754.25
Epoch: 38, E